In [2]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f1.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.2746,AUC: 0.9203,AP: 0.9486
Epoch: 002, Loss: 0.1741,AUC: 0.9283,AP: 0.9534
Epoch: 003, Loss: 0.1662,AUC: 0.9312,AP: 0.9550
Epoch: 004, Loss: 0.1638,AUC: 0.9375,AP: 0.9587
Epoch: 005, Loss: 0.1631,AUC: 0.9379,AP: 0.9591
Epoch: 006, Loss: 0.1629,AUC: 0.9409,AP: 0.9611
Epoch: 007, Loss: 0.1597,AUC: 0.9391,AP: 0.9604
Epoch: 008, Loss: 0.1595,AUC: 0.9429,AP: 0.9627
Epoch: 009, Loss: 0.1587,AUC: 0.9425,AP: 0.9628
Epoch: 010, Loss: 0.1582,AUC: 0.9458,AP: 0.9649
Epoch: 011, Loss: 0.1577,AUC: 0.9425,AP: 0.9634
Epoch: 012, Loss: 0.1564,AUC: 0.9431,AP: 0.9638
Epoch: 013, Loss: 0.1552,AUC: 0.9494,AP: 0.9679
Epoch: 014, Loss: 0.1557,AUC: 0.9458,AP: 0.9661
Epoch: 015, Loss: 0.1546,AUC: 0.9445,AP: 0.9653
Epoch: 016, Loss: 0.1534,AUC: 0.9455,AP: 0.9660
Epoch: 017, Loss: 0.1524,AUC: 0.9436,AP: 0.9654
Epoch: 018, Loss: 0.1538,AUC: 0.9466,AP: 0.9671
Epoch: 019, Loss: 0.1537,AUC: 0.9470,AP: 0.9672
Epoch: 020, Loss: 0.1538,AUC: 0.9469,AP: 0.9673
Epoch: 021, Loss: 0.1518,AUC: 0.9477,AP:

In [4]:
z1 = model(train_data.x, train_data.edge_index)
z1.shape

torch.Size([27045, 128])

In [5]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f2.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.3251,AUC: 0.9138,AP: 0.9483
Epoch: 002, Loss: 0.1999,AUC: 0.9209,AP: 0.9516
Epoch: 003, Loss: 0.1874,AUC: 0.9256,AP: 0.9543
Epoch: 004, Loss: 0.1852,AUC: 0.9298,AP: 0.9570
Epoch: 005, Loss: 0.1839,AUC: 0.9312,AP: 0.9583
Epoch: 006, Loss: 0.1838,AUC: 0.9313,AP: 0.9586
Epoch: 007, Loss: 0.1795,AUC: 0.9335,AP: 0.9599
Epoch: 008, Loss: 0.1808,AUC: 0.9319,AP: 0.9591
Epoch: 009, Loss: 0.1805,AUC: 0.9325,AP: 0.9596
Epoch: 010, Loss: 0.1798,AUC: 0.9300,AP: 0.9586
Epoch: 011, Loss: 0.1772,AUC: 0.9309,AP: 0.9593
Epoch: 012, Loss: 0.1771,AUC: 0.9314,AP: 0.9598
Epoch: 013, Loss: 0.1769,AUC: 0.9337,AP: 0.9611
Epoch: 014, Loss: 0.1750,AUC: 0.9343,AP: 0.9617
Epoch: 015, Loss: 0.1742,AUC: 0.9368,AP: 0.9630
Epoch: 016, Loss: 0.1763,AUC: 0.9373,AP: 0.9632
Epoch: 017, Loss: 0.1745,AUC: 0.9341,AP: 0.9616
Epoch: 018, Loss: 0.1749,AUC: 0.9340,AP: 0.9617
Epoch: 019, Loss: 0.1732,AUC: 0.9363,AP: 0.9628
Epoch: 020, Loss: 0.1728,AUC: 0.9362,AP: 0.9629
Epoch: 021, Loss: 0.1742,AUC: 0.9387,AP:

In [7]:
z2 = model(train_data.x, train_data.edge_index)
z2.shape

torch.Size([27045, 128])

In [8]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f3.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.4099,AUC: 0.9108,AP: 0.9456
Epoch: 002, Loss: 0.2508,AUC: 0.9194,AP: 0.9495
Epoch: 003, Loss: 0.2436,AUC: 0.9197,AP: 0.9495
Epoch: 004, Loss: 0.2402,AUC: 0.9256,AP: 0.9528
Epoch: 005, Loss: 0.2371,AUC: 0.9253,AP: 0.9530
Epoch: 006, Loss: 0.2350,AUC: 0.9283,AP: 0.9548
Epoch: 007, Loss: 0.2322,AUC: 0.9324,AP: 0.9572
Epoch: 008, Loss: 0.2318,AUC: 0.9293,AP: 0.9560
Epoch: 009, Loss: 0.2321,AUC: 0.9316,AP: 0.9573
Epoch: 010, Loss: 0.2305,AUC: 0.9320,AP: 0.9578
Epoch: 011, Loss: 0.2280,AUC: 0.9326,AP: 0.9586
Epoch: 012, Loss: 0.2277,AUC: 0.9286,AP: 0.9566
Epoch: 013, Loss: 0.2280,AUC: 0.9343,AP: 0.9598
Epoch: 014, Loss: 0.2253,AUC: 0.9355,AP: 0.9607
Epoch: 015, Loss: 0.2256,AUC: 0.9340,AP: 0.9603
Epoch: 016, Loss: 0.2257,AUC: 0.9353,AP: 0.9611
Epoch: 017, Loss: 0.2251,AUC: 0.9391,AP: 0.9632
Epoch: 018, Loss: 0.2239,AUC: 0.9363,AP: 0.9619
Epoch: 019, Loss: 0.2253,AUC: 0.9369,AP: 0.9625
Epoch: 020, Loss: 0.2242,AUC: 0.9329,AP: 0.9605
Epoch: 021, Loss: 0.2240,AUC: 0.9342,AP:

In [9]:
z3 = model(train_data.x, train_data.edge_index)
z3.shape

torch.Size([27045, 128])

In [10]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f4.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.3318,AUC: 0.9002,AP: 0.9377
Epoch: 002, Loss: 0.2130,AUC: 0.9049,AP: 0.9398
Epoch: 003, Loss: 0.2066,AUC: 0.9087,AP: 0.9417
Epoch: 004, Loss: 0.2010,AUC: 0.9131,AP: 0.9441
Epoch: 005, Loss: 0.2009,AUC: 0.9089,AP: 0.9424
Epoch: 006, Loss: 0.2011,AUC: 0.9079,AP: 0.9421
Epoch: 007, Loss: 0.1993,AUC: 0.9100,AP: 0.9434
Epoch: 008, Loss: 0.1983,AUC: 0.9124,AP: 0.9451
Epoch: 009, Loss: 0.1960,AUC: 0.9145,AP: 0.9467
Epoch: 010, Loss: 0.1952,AUC: 0.9105,AP: 0.9452
Epoch: 011, Loss: 0.1957,AUC: 0.9098,AP: 0.9452
Epoch: 012, Loss: 0.1932,AUC: 0.9078,AP: 0.9444
Epoch: 013, Loss: 0.1930,AUC: 0.9116,AP: 0.9464
Epoch: 014, Loss: 0.1915,AUC: 0.9141,AP: 0.9478
Epoch: 015, Loss: 0.1917,AUC: 0.9107,AP: 0.9466
Epoch: 016, Loss: 0.1909,AUC: 0.9149,AP: 0.9487
Epoch: 017, Loss: 0.1913,AUC: 0.9179,AP: 0.9504
Epoch: 018, Loss: 0.1903,AUC: 0.9141,AP: 0.9488
Epoch: 019, Loss: 0.1903,AUC: 0.9128,AP: 0.9484
Epoch: 020, Loss: 0.1902,AUC: 0.9201,AP: 0.9521
Epoch: 021, Loss: 0.1888,AUC: 0.9154,AP:

In [11]:
z4 = model(train_data.x, train_data.edge_index)
z4.shape

torch.Size([27045, 128])

In [12]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f5.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.3782,AUC: 0.9128,AP: 0.9477
Epoch: 002, Loss: 0.2197,AUC: 0.9235,AP: 0.9531
Epoch: 003, Loss: 0.2096,AUC: 0.9263,AP: 0.9543
Epoch: 004, Loss: 0.2078,AUC: 0.9293,AP: 0.9562
Epoch: 005, Loss: 0.2048,AUC: 0.9304,AP: 0.9571
Epoch: 006, Loss: 0.2013,AUC: 0.9333,AP: 0.9588
Epoch: 007, Loss: 0.2016,AUC: 0.9317,AP: 0.9583
Epoch: 008, Loss: 0.1991,AUC: 0.9344,AP: 0.9599
Epoch: 009, Loss: 0.1996,AUC: 0.9337,AP: 0.9600
Epoch: 010, Loss: 0.1974,AUC: 0.9332,AP: 0.9598
Epoch: 011, Loss: 0.1961,AUC: 0.9352,AP: 0.9612
Epoch: 012, Loss: 0.1977,AUC: 0.9349,AP: 0.9613
Epoch: 013, Loss: 0.1957,AUC: 0.9353,AP: 0.9616
Epoch: 014, Loss: 0.1968,AUC: 0.9382,AP: 0.9631
Epoch: 015, Loss: 0.1947,AUC: 0.9377,AP: 0.9630
Epoch: 016, Loss: 0.1943,AUC: 0.9400,AP: 0.9643
Epoch: 017, Loss: 0.1947,AUC: 0.9393,AP: 0.9640
Epoch: 018, Loss: 0.1922,AUC: 0.9393,AP: 0.9642
Epoch: 019, Loss: 0.1932,AUC: 0.9383,AP: 0.9639
Epoch: 020, Loss: 0.1923,AUC: 0.9400,AP: 0.9649
Epoch: 021, Loss: 0.1915,AUC: 0.9356,AP:

In [13]:
z5 = model(train_data.x, train_data.edge_index)
z5.shape

torch.Size([27045, 128])

In [14]:
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f6.pt')
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045
data['x'] = torch.randn((27045, 100))
data = data.to(device, 'x', 'edge_index', 'edge_attr')
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)
assert train_data.edge_index.max() < train_data.num_nodes  # !!!!!!!!
train_data = train_data.cpu()  # 采样使用cpu
loader = LinkNeighborLoader(
    data=train_data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)
model = GraphSAGE(
    in_channels=train_data.num_node_features,
    hidden_channels=128,
    num_layers=2,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)


def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        pred = (h[batch.edge_label_index[0]] * h[batch.edge_label_index[1]]).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    data = test_data.to(device)
    z = model(data.x, data.edge_index)
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Epoch: 001, Loss: 0.2513,AUC: 0.9209,AP: 0.9487
Epoch: 002, Loss: 0.1618,AUC: 0.9188,AP: 0.9479
Epoch: 003, Loss: 0.1542,AUC: 0.9256,AP: 0.9518
Epoch: 004, Loss: 0.1516,AUC: 0.9302,AP: 0.9548
Epoch: 005, Loss: 0.1499,AUC: 0.9275,AP: 0.9536
Epoch: 006, Loss: 0.1509,AUC: 0.9267,AP: 0.9535
Epoch: 007, Loss: 0.1491,AUC: 0.9283,AP: 0.9549
Epoch: 008, Loss: 0.1458,AUC: 0.9322,AP: 0.9574
Epoch: 009, Loss: 0.1480,AUC: 0.9270,AP: 0.9549
Epoch: 010, Loss: 0.1455,AUC: 0.9252,AP: 0.9541
Epoch: 011, Loss: 0.1457,AUC: 0.9289,AP: 0.9563
Epoch: 012, Loss: 0.1460,AUC: 0.9256,AP: 0.9548
Epoch: 013, Loss: 0.1453,AUC: 0.9278,AP: 0.9561
Epoch: 014, Loss: 0.1442,AUC: 0.9268,AP: 0.9558
Epoch: 015, Loss: 0.1446,AUC: 0.9295,AP: 0.9576
Epoch: 016, Loss: 0.1420,AUC: 0.9286,AP: 0.9573
Epoch: 017, Loss: 0.1424,AUC: 0.9313,AP: 0.9590
Epoch: 018, Loss: 0.1412,AUC: 0.9279,AP: 0.9571
Epoch: 019, Loss: 0.1439,AUC: 0.9289,AP: 0.9577
Epoch: 020, Loss: 0.1421,AUC: 0.9300,AP: 0.9585
Epoch: 021, Loss: 0.1414,AUC: 0.9310,AP:

In [15]:
z6 = model(train_data.x, train_data.edge_index)
z6.shape

torch.Size([27045, 128])

In [16]:
z_1_6 = torch.cat([z1, z2, z3, z4, z5, z6], 1)
z_1_6.shape

torch.Size([27045, 768])

In [17]:
torch.save(z_1_6, '/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/z_1_6_gs.pt')

In [24]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

# z_1_6=torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/z_1_6.pt')
pyg.seed_everything(3407)
device = 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f7.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = 27045

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.0,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

print(train_data.edge_index.shape)


@torch.no_grad()
def test():
    data = train_data
    z = z_1_6  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


AUC, AP = test()
print(f'AUC: {AUC:.4f},AP: {AP:.4f}')

torch.Size([2, 8538])
AUC: 0.8580,AP: 0.9205
